In [ ]:
from google.colab import files
import shutil
import zipfile
import os

# Create a folder to store the uploaded files
folder_name = 'uploaded_folder'
shutil.rmtree(folder_name, ignore_errors=True)
os.makedirs(folder_name)
uploaded = files.upload()

zip_ref = zipfile.ZipFile(list(uploaded.keys())[0], 'r')
zip_ref.extractall(folder_name)
zip_ref.close()

In [ ]:
import zipfile

# Specify the path of the zipped file
zip_path = '/content/Training.zip'

# Specify the directory to extract the contents
extract_dir = '/content/uploaded_folder/Train'

# Open the zipped file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the files to the extract_dir
    zip_ref.extractall(extract_dir)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/uploaded_folder/Train/Training"
test_dir = "/content/uploaded_folder/test1/Test"

# Load the dataset
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical')
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(131, activation='softmax')  # Assuming 131 classes
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator)

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("fruit_model2.tflite", "wb").write(tflite_model)

from google.colab import files

# Assuming your model's filename is 'model.tflite'
files.download('fruit_model2.tflite')


added a Dropout layer after each MaxPooling2D layer in your model, which randomly drops out 25% of the input units in each forward pass.
I've also added L2 regularization to the Dense layer. Additionally, I've included an EarlyStopping callback that stops training when the validation loss hasn't improved in 3

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
#Load the dataset with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical')

# Define the model with dropout and regularization
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),  # dropout layer
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),  # dropout layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=l2(0.01)),  # L2 regularization
    tf.keras.layers.Dense(131, activation='softmax')  # Assuming 131 classes
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping])  # early stopping


In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("fruit_model3.tflite", "wb").write(tflite_model)

from google.colab import files

# Assuming your model's filename is 'model.tflite'
files.download('fruit_model3.tflite')